# Data Engineer - Technical Assessment

In this section of the interview at Beyond Finance, you will be assessed on your ability to perform several Data Engineering tasks. To perform well on this task, you will demonstate competence in the following areas:

* preprocessing data to prepare for a database load
* understanding entity relationships in a database
* merging data from different tables
* filtering data to relevant subsets
* calculating aggregations and descriptive statistics

It will be pretty difficult to complete all questions in the allotted time. Your goal is not to speed through the answers, but to come up with answers that demonstrate your knowledge. It's more about your thought process and logic than getting the right answer or your code.


## Getting Started

This exercise will be broken into 2 parts
1. Data Processing
2. Data Wrangling

### Data Processing
In this section you will take files from the ./raw_data/ subfolders, combine them into a single newline-delimited `json.gz` file per subfolder, and place that CSV file in a ./processed_data/ directory. You may have to do some light investigation into the data files to understand their file formats and delimiters

**Example**

Files
- ./raw_data/tracks/tracks_0.csv
- ./raw_data/tracks/tracks_1.json
- ./raw_data/tracks/tracks_2.csv
- etc... 

should be combined into a single file ./processed_data/tracks.json.gz

**What we look for**

- Can you handle all subfolders in a single pass over the raw data files?
- How can you limit memory consumption? (hint `chunksize`)

### Data Wrangling
For this section, we'll pretend you loaded the raw data plus additional tables into a small SQLite database containing roughly a dozen tables. **We've provided this database for you so don't worry about loading it yourself**. If you are not familiar with the SQLite database, it uses a fairly complete and standard SQL syntax, though does not many advanced analytics functions. Consider it just a remote datastore for storing and retrieving data from. 

![](db-diagram.png)

## Data Processing (40 minutes)

In [2]:
import pandas as pd 

!pip install memory_profiler
%load_ext memory_profiler

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 7.2 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If yo

In [4]:
%%memit
# ... your code here

peak memory: 69.55 MiB, increment: 1.79 MiB


In [6]:
pip install ipython-sql


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.0/270.0 kB 2.5 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com

## Data Wrangling (20 minutes)

In [22]:
%load_ext sql 
%sql sqlite:///db/sqlite/chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
import sqlite3

con = sqlite3.connect("db/sqlite/chinook.db")

In [19]:
cursor = con.cursor()


In [21]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:", [table[0] for table in tables])

Tables in the database: ['albums', 'sqlite_sequence', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks', 'sqlite_stat1']


In [26]:
import pandas as pd

df = pd.read_sql("SELECT * FROM customers LIMIT 5;", con)
print(df)

   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None   
2                                              None   
3                                              None   
4                                  JetBrains s.r.o.   

                           Address                 City State         Country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP          Brazil   
1          Theodor-Heuss-Straße 34            Stuttgart  None         Germany   
2                1498 rue Bélanger             Montréal    QC          Canada   
3                 Ullevålsveien 14                 Oslo  None          Norway   
4                

### 1. How many different customers are there?

In [25]:
cursor.execute("SELECT COUNT(DISTINCT CustomerId) FROM customers;")
unique_customers = cursor.fetchone()[0]

print(f"Total unique customers: {unique_customers}")


Total unique customers: 59


### 2. How long is the longest track in minutes?

In [29]:

df2 = pd.read_sql("SELECT * FROM tracks LIMIT 5;", con)
print(df2)
cursor.execute("SELECT MAX((Milliseconds/1000)/60) FROM tracks;")
longest_track = cursor.fetchone()[0]

print(f"Longest track: {longest_track}")


   TrackId                                     Name  AlbumId  MediaTypeId  \
0        1  For Those About To Rock (We Salute You)        1            1   
1        2                        Balls to the Wall        2            2   
2        3                          Fast As a Shark        3            2   
3        4                        Restless and Wild        3            2   
4        5                     Princess of the Dawn        3            2   

   GenreId                                           Composer  Milliseconds  \
0        1          Angus Young, Malcolm Young, Brian Johnson        343719   
1        1                                               None        342562   
2        1  F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...        230619   
3        1  F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...        252051   
4        1                         Deaffy & R.A. Smith-Diesel        375418   

      Bytes  UnitPrice  
0  11170334       0.99  
1   5510424 

In [34]:
cursor.execute("""
    SELECT Name AS track_name, ROUND(Milliseconds / 60000.0, 2) AS length_minutes
    FROM tracks
    ORDER BY Milliseconds DESC
    LIMIT 1;
""")

# Fetch result
longest_track = cursor.fetchone()
print(f"The longest track is '{longest_track[0]}' with a length of {longest_track[1]} minutes.")

The longest track is 'Occupation / Precipice' with a length of 88.12 minutes.


In [35]:
cursor = con.cursor()

query = """
    SELECT Name AS track_name, Milliseconds, ROUND(Milliseconds / 60000.0, 2) AS length_minutes
    FROM tracks
    ORDER BY Milliseconds DESC
    LIMIT 5;
"""
df = pd.read_sql(query, con)
print(df)


                    track_name  Milliseconds  length_minutes
0       Occupation / Precipice       5286953           88.12
1      Through a Looking Glass       5088838           84.81
2  Greetings from Earth, Pt. 1       2960293           49.34
3      The Man With Nine Lives       2956998           49.28
4  Battlestar Galactica, Pt. 2       2956081           49.27


In [38]:
query = """
    SELECT DISTINCT a.Title AS album, ar.Name AS artist
    FROM tracks t
    JOIN albums a ON t.AlbumId = a.AlbumId
    JOIN artists ar ON a.ArtistId = ar.ArtistId
    WHERE t.Name IN (
    'Occupation / Precipice',
    'Through a Looking Glass',
    'Greetings from Earth, Pt. 1',
    'The Man With Nine Lives',
    'Battlestar Galactica, Pt. 2'
);
"""
df3 = pd.read_sql(query, con)
print(df3)

                                      album                          artist
0            Battlestar Galactica, Season 3            Battlestar Galactica
1                            Lost, Season 3                            Lost
2  Battlestar Galactica (Classic), Season 1  Battlestar Galactica (Classic)


In [30]:
cursor.execute("SELECT ROUND(MAX(milliseconds) / 60000.0, 2) FROM tracks;")
longest_track = cursor.fetchone()[0]

print(f"The longest track is {longest_track} minutes.")

The longest track is 88.12 minutes.


### 3. Which genre has the shortest average track length?

In [31]:
cursor.execute("""
    SELECT g.Name AS genre, 
           ROUND(AVG(t.Milliseconds) / 60000.0, 2) AS avg_length_minutes
    FROM tracks t
    JOIN genres g ON t.GenreId = g.GenreId
    GROUP BY g.Name
    ORDER BY avg_length_minutes ASC
    LIMIT 1;
""")

# Fetch result
shortest_genre = cursor.fetchone()
print(f"The genre with the shortest average track length is '{shortest_genre[0]}' with an average length of {shortest_genre[1]} minutes.")


The genre with the shortest average track length is 'Rock And Roll' with an average length of 2.24 minutes.


### 4. Which artist shows up in the most playlists?

In [32]:
cursor.execute("""
    SELECT ar.Name AS artist, COUNT(DISTINCT pt.PlaylistId) AS playlist_count
    FROM playlist_track pt
    JOIN tracks t ON pt.TrackId = t.TrackId
    JOIN albums al ON t.AlbumId = al.AlbumId
    JOIN artists ar ON al.ArtistId = ar.ArtistId
    GROUP BY ar.ArtistId
    ORDER BY playlist_count DESC
    LIMIT 1;
""")

# Fetch result
most_playlisted_artist = cursor.fetchone()
print(f"The artist appearing in the most playlists is '{most_playlisted_artist[0]}' with {most_playlisted_artist[1]} playlists.")

The artist appearing in the most playlists is 'Eugene Ormandy' with 7 playlists.


### 5. What album had the most purchases?

In [33]:
cursor.execute("""
    SELECT al.Title AS album, ar.Name AS artist, COUNT(ii.InvoiceId) AS purchase_count
    FROM invoice_items ii
    JOIN tracks t ON ii.TrackId = t.TrackId
    JOIN albums al ON t.AlbumId = al.AlbumId
    JOIN artists ar ON al.ArtistId = ar.ArtistId
    GROUP BY al.AlbumId
    ORDER BY purchase_count DESC
    LIMIT 1;
""")

# Fetch result
top_album = cursor.fetchone()
print(f"The most purchased album is '{top_album[0]}' by {top_album[1]} with {top_album[2]} purchases.")

The most purchased album is 'Minha Historia' by Chico Buarque with 27 purchases.


In [51]:
query = """
SELECT 
    c.FirstName || ' ' || c.LastName AS CustomerName
FROM invoices i
JOIN customers c ON i.CustomerId = c.CustomerId
GROUP BY c.CustomerId, CustomerName
ORDER BY SUM(i.Total) DESC
LIMIT 1;
"""

cursor.execute(query)

top_customer = cursor.fetchone()

if top_customer:
    customer_name = top_customer
    print(f"Highest-Spending Customer: {customer_name}")
else:
    print(" No results found.")



Highest-Spending Customer: ('Helena Holý',)


In [48]:
query = """
SELECT COUNT(*) 
FROM (
    SELECT c.CustomerId
    FROM invoices i
    JOIN customers c ON i.CustomerId = c.CustomerId
    GROUP BY c.CustomerId
    HAVING SUM(i.Total) > 40
) AS high_spending_customers;
"""

cursor.execute(query)

customer_count = cursor.fetchone()[0]

In [49]:
print(f"Number of customers with sales > $40: {customer_count}")


Number of customers with sales > $40: 14
